In [1]:
import numpy as np 
import pandas as pd
from sklearn.neighbors.kde import KernelDensity
from sklearn.metrics import precision_recall_fscore_support as prf, accuracy_score
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn

In [2]:
data = pd.read_csv("kddcup.data_10_percent_corrected", header=None,names=['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'type'])

In [3]:
proportions = data["type"].value_counts()
print(proportions)

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: type, dtype: int64


In [4]:
data.loc[data["type"] == "normal.", 'type'] = 0
data.loc[data["type"] == "guess_passwd.", 'type'] = 1
data.loc[data["type"] == "smurf.", 'type'] = 1
data.loc[data["type"] == "neptune.", 'type'] = 1
data.loc[data["type"] == "back.", 'type'] = 1
data.loc[data["type"] == "satan.", 'type'] = 1
data.loc[data["type"] == "ipsweep.", 'type'] = 1
data.loc[data["type"] == "portsweep.", 'type'] = 1
data.loc[data["type"] == "warezclient.", 'type'] = 1
data.loc[data["type"] == "teardrop.", 'type'] = 1
data.loc[data["type"] == "pod.", 'type'] = 1
data.loc[data["type"] == "nmap.", 'type'] = 1
data.loc[data["type"] == "buffer_overflow.", 'type'] = 1
data.loc[data["type"] == "land.", 'type'] = 1
data.loc[data["type"] == "neptune.", 'type'] = 1
data.loc[data["type"] == "warezmaster.", 'type'] = 1
data.loc[data["type"] == "imap.", 'type'] = 1
data.loc[data["type"] == "rootkit.", 'type'] = 1
data.loc[data["type"] == "loadmodule.", 'type'] = 1
data.loc[data["type"] == "ftp_write.", 'type'] = 1
data.loc[data["type"] == "multihop.", 'type'] = 1
data.loc[data["type"] == "phf.", 'type'] = 1
data.loc[data["type"] == "perl.", 'type'] = 1
data.loc[data["type"] == "spy.", 'type'] = 1

In [5]:
proportions = data["type"].value_counts()
print(proportions)

1    396743
0     97278
Name: type, dtype: int64


In [6]:
one_hot_protocol = pd.get_dummies(data["protocol_type"])
one_hot_service = pd.get_dummies(data["service"])
one_hot_flag = pd.get_dummies(data["flag"])

data = data.drop("protocol_type",axis=1)
data = data.drop("service",axis=1)
data = data.drop("flag",axis=1)
    
data = pd.concat([one_hot_protocol, one_hot_service,one_hot_flag, data],axis=1)

In [7]:
cols_to_norm = ["duration", "src_bytes", "dst_bytes", "wrong_fragment", "urgent", 
            "hot", "num_failed_logins", "num_compromised", "num_root", 
            "num_file_creations", "num_shells", "num_access_files", "count", "srv_count", 
            "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate", 
            "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate", 
            "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", 
            "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate" ]

# data.loc[:, cols_to_norm] = (data[cols_to_norm] - data[cols_to_norm].mean()) / data[cols_to_norm].std()
min_cols = data.loc[data["type"]==1 , cols_to_norm].min()
max_cols = data.loc[data["type"]==1 , cols_to_norm].max()

data.loc[:, cols_to_norm] = (data[cols_to_norm] - min_cols) / (max_cols - min_cols)

In [8]:
np.savez_compressed("kdd_cup",kdd=data.as_matrix())

C:\ProgramData\Anaconda2\envs\python35\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
